In [91]:
import numpy as np
import pandas as pd
import numba as nb
from typing import List
import math
from fractions import Fraction

from numpy import ndarray

np.seterr(divide='ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
U = {
    (4, 'Маг'): {
        'Z': 5,
        'D': 3,
        'K': 4,
    },
    (5, 'Рыцарь'): {
        'Z': 4,
        'D': 5,
        'K': 7,
    },
    (50, 'Дракон'): {
        'Z': 50,
        'D': 35,
        'K': 45,
    }
}

R = {
    'Z': 450,
    'D': 300,
    'K': 500,
}

# Тупой перебор

In [3]:
s = np.array([u[0] for u in U]).astype('float64')
p = np.array([[Uvv for Uvk, Uvv in Uv.items()] for Uk, Uv in U.items()]).astype('float64')
r = np.array([Rv for Rk, Rv in R.items()]).astype('float64')

print(s)
print(p)
print(r)

[ 4.  5. 50.]
[[ 5.  3.  4.]
 [ 4.  5.  7.]
 [50. 35. 45.]]
[450. 300. 500.]


In [4]:
@nb.njit('float64[:](float64[:], float64[:,:], float64[:])', parallel=True)
def dump(powers, prices, resources):
    best = 0
    best_x = np.zeros_like(powers)

    # maxes = np.floor(np.min(1 / np.divide(prices, resources.T), axis=1))
    maxes = np.floor(np.array([np.min(r) for r in 1 / np.divide(prices, resources.T)]))

    for X1 in range(0, int(maxes[0])):
        for X2 in range(0, int(maxes[1])):
            # print(resources - (prices[0] * X1 + prices[1] * X2), prices[2])
            x = np.array(
                [X1, X2, np.floor(np.min(np.divide(resources - (prices[0] * X1 + prices[1] * X2), prices[2])))])
            strength = np.sum(np.multiply(powers, x))
            if strength > best:
                best = strength
                best_x = x
                # print(strength, x)
    return best_x


d = dump(s, p, r)
d, np.sum(np.multiply(s, d)), np.sum(np.multiply(p.T, d), axis=1), r - np.sum(np.multiply(p.T, d), axis=1)

(array([5., 1., 8.]),
 np.float64(425.0),
 array([429., 300., 387.]),
 array([ 21.,   0., 113.]))

In [5]:
@nb.njit('(float64[:], float64[:,:], float64[:])')
def test(s, p, r):
    for i in range(100):
        dump(s, p, r)


test(s, p, r)

Интересно, что процессор занят в 100%, но при этом потребление в ватах как в простое. Очевидно, что процессор вместо дела сидит и ждет когда ему освободят данные.

In [6]:
# @nb.njit('(float64[:], float64[:,:], float64[:])', parallel=True)
# def test(s, p, r):
#     for i in range(100):
#         dump(s, p, r)
#
#
# # test(s, p, r)
# test.parallel_diagnostics(level=4)

In [7]:
@nb.njit('float64[:](float64[:], float64[:,:], float64[:])')
def dump(powers, prices, resources):
    best = 0
    best_x = np.zeros_like(powers)

    # maxes = np.floor(np.min(1 / np.divide(prices, resources.T), axis=1))
    maxes = np.floor(np.array([np.min(r) for r in 1 / np.divide(prices, resources.T)]))

    for X1 in range(0, int(maxes[0])):
        for X2 in range(0, int(maxes[1])):
            # print(resources - (prices[0] * X1 + prices[1] * X2), prices[2])
            x = np.array(
                [X1, X2, np.floor(np.min(np.divide(resources - (prices[0] * X1 + prices[1] * X2), prices[2])))])
            strength = np.sum(np.multiply(powers, x))
            if strength > best:
                best = strength
                best_x = x
                # print(strength, x)
    return best_x


d = dump(s, p, r)
d, np.sum(np.multiply(s, d)), np.sum(np.multiply(p.T, d), axis=1), r - np.sum(np.multiply(p.T, d), axis=1)

(array([5., 1., 8.]),
 np.float64(425.0),
 array([429., 300., 387.]),
 array([ 21.,   0., 113.]))

In [8]:
@nb.njit('(float64[:], float64[:,:], float64[:])')
def test(s, p, r):
    for i in range(1000):
        dump(s, p, r)


test(s, p, r)

In [9]:
def dump(powers, prices, resources):
    best = 0
    best_x = np.zeros_like(powers)

    maxes = np.floor(np.min(1 / np.divide(prices, resources.T), axis=1))
    # maxes = np.floor(np.array([np.min(r) for r in 1 / np.divide(prices, resources.T)]))

    for X1 in range(0, int(maxes[0])):
        for X2 in range(0, int(maxes[1])):
            # print(resources - (prices[0] * X1 + prices[1] * X2), prices[2])
            x = np.array(
                [X1, X2, np.floor(np.min(np.divide(resources - (prices[0] * X1 + prices[1] * X2), prices[2])))])
            strength = np.sum(np.multiply(powers, x))
            if strength > best:
                best = strength
                best_x = x
                # print(strength, x)
    return best_x


d = dump(s, p, r)
d, np.sum(np.multiply(s, d)), np.sum(np.multiply(p.T, d), axis=1), r - np.sum(np.multiply(p.T, d), axis=1)

(array([5., 1., 8.]),
 np.float64(425.0),
 array([429., 300., 387.]),
 array([ 21.,   0., 113.]))

In [10]:
def test(s, p, r):
    for i in range(1000):
        dump(s, p, r)


test(s, p, r)

# Метод Гомори

In [113]:
def gomori_max(table: np.ndarray, columns: List[str] = None, rows: List[str] = None) -> dict:
    iteration = 0
    columns = ['free'] + [f'X{i}' for i in range(1, table.shape[1])] if columns is None else columns
    rows = [f'X{i + table.shape[1]}' for i in range(table.shape[0] - 1)] + ['F'] if rows is None else rows
    table = np.array(table).copy()
    table = table + Fraction()
    prev_leading_col = None
    r = np.vectorize(round)

    start_table = pd.DataFrame(table, columns=columns, index=rows)
    # max_iter = math.e ** np.sum(start_table.shape)

    print(iteration)
    print(start_table)

    def step(table: np.ndarray, columns: List[str], rows: List[str], leading_row=None, leading_col=None, optimization=True) -> dict:
        nonlocal iteration, prev_leading_col

        iteration += 1
        print("iteration:", iteration)
        # print(pd.DataFrame(table, columns=columns, index=rows))

        table2 = table.copy()
        # if prev_leading_col is not None:
        #     table2[0, prev_leading_col] = 0

        if leading_col is None:
            leading_col = np.where(table2[-1, 1:] <= 0)[0][0] + 1 if optimization and sum(table2[-1, 1:] <= 0) == 1 \
                else np.abs(table2[:-1, 1:]).argmax() % (table.shape[1] - 1) + 1

        prev_leading_col = leading_col

        if leading_row is None:
            col = np.divide(table2.T[0, :-1], table2.T[leading_col, :-1])
            col[np.logical_and(table2.T[0, :-1] >= 0, table2.T[leading_col, :-1] <= 0)] = np.inf
            leading_row = np.abs(col).argmin()

            # print("col:", col)

        table2[0] = table[leading_row] / table[leading_row, leading_col]
        table2[0, leading_col] = 1 / table[leading_row, leading_col]
        auxiliary = -table.T[leading_col]
        auxiliary = np.delete(auxiliary, leading_row)
        table = np.delete(table, leading_row, axis=0)
        table.T[leading_col] = 0

        for i in range(auxiliary.shape[0]):
            table2[i + 1] = table2[0] * auxiliary[i] + table[i]

        table = table2

        changed_col = columns[leading_col]
        columns[leading_col] = rows.pop(leading_row)
        rows.insert(0, changed_col)

        print("leading col, row:", leading_col, leading_row)
        print(pd.DataFrame(table, columns=columns, index=rows))
        print(auxiliary)

        return table

    while np.sum(table[-1] >= 0) < table.shape[1]:
        table = step(table, columns, rows)

    target_row = table[np.isin(rows, start_table.columns)]
    print(target_row)
    target_row = target_row[np.abs(target_row[:, 0] - r(target_row[:, 0])) != 0]

    while len(target_row) > 0:
        target_row = target_row - np.floor(target_row)
        target_row = target_row[target_row[:, 0].argmax()]
        rows.insert(len(rows) - 1, f'A{np.sum(table.shape) - np.sum(start_table.shape) + 1}')
        table = np.insert(table, [len(rows) - 2], -target_row, axis=0)
        print("iteration:", iteration + 0.5)
        print(pd.DataFrame(table, columns=columns, index=rows))
        table = step(table, columns, rows, optimization=False)
        while np.sum(table[-1] >= 0) < table.shape[1]:
            table = step(table, columns, rows)
        target_row = table[np.isin(rows, start_table.columns)]
        target_row = target_row[np.abs(target_row[:, 0] - r(target_row[:, 0])) != 0]

    print("iterations:", iteration)
    values = pd.DataFrame(table, columns=columns, index=rows).iloc[:, 0].round().astype(int).to_dict()
    for i in range(1, len(columns)):
        values[columns[i]] = 0
    return values


gomori_max(np.array([
    [6, 2, 1],
    [5, 1, 4],
    [0, -3, -2],
]).astype(float))

0
   free   X1   X2
X3  6.0  2.0  1.0
X4  5.0  1.0  4.0
F   0.0 -3.0 -2.0
iteration: 1
leading col, row: 2 1
    free    X1    X4
X2  1.25  0.25  0.25
X3  4.75  1.75 -0.25
F    2.5  -2.5   0.5
[-1.0 2.0]
iteration: 2
leading col, row: 1 1
        free        X3        X4
X1  2.714286  0.571429 -0.142857
X2  0.571429 -0.142857  0.285714
F   9.285714  1.428571  0.142857
[-0.25 2.5]
[[2.7142857142857144 0.5714285714285714 -0.14285714285714285]
 [0.5714285714285714 -0.14285714285714285 0.2857142857142857]]
iteration: 2.5
        free        X3        X4
X1  2.714286  0.571429 -0.142857
X2  0.571429 -0.142857  0.285714
A1 -0.714286 -0.571429 -0.857143
F   9.285714  1.428571  0.142857
iteration: 3
leading col, row: 2 2
        free        X3        A1
X4  0.833333  0.666667 -1.166667
X1  2.833333  0.666667 -0.166667
X2  0.333333 -0.333333  0.333333
F   9.166667  1.333333  0.166667
[0.14285714285714285 -0.2857142857142857 -0.1428571428571429]
iteration: 3.5
        free        X3        A1
X4

{'A1': 1, 'X4': 2, 'X1': 3, 'A2': 0, 'F': 9, 'X3': 0, 'X2': 0}

In [111]:
Xs = [[Rv] + [Uv[Rk] for Uk, Uv in U.items()] for Rk, Rv in R.items()] + [[0] + [-Uk[0] for Uk, Uv in U.items()]]
np.array(Xs)

array([[450,   5,   4,  50],
       [300,   3,   5,  35],
       [500,   4,   7,  45],
       [  0,  -4,  -5, -50]])

In [112]:
j = gomori_max(Xs, rows=['Z', 'D', 'K', 'F'], columns=['free', 'X1', 'X2', 'X3'])
j = {i: j[i] for i in ['Z', 'D', 'K', 'F'] + ['X1', 'X2', 'X3'] + ['F']}
j

0
  free  X1  X2   X3
Z  450   5   4   50
D  300   3   5   35
K  500   4   7   45
F    0  -4  -5  -50
iteration: 1
leading col, row: 3 1
      free    X1     X2      D
X3    60/7  3/35    1/7   1/35
Z    150/7   5/7  -22/7  -10/7
K    800/7   1/7    4/7   -9/7
F   3000/7   2/7   15/7   10/7
[Fraction(-50, 1) Fraction(-45, 1) Fraction(50, 1)]
[[Fraction(60, 7) Fraction(3, 35) Fraction(1, 7) Fraction(1, 35)]]
iteration: 1.5
      free     X1     X2      D
X3    60/7   3/35    1/7   1/35
Z    150/7    5/7  -22/7  -10/7
K    800/7    1/7    4/7   -9/7
A1    -4/7  -3/35   -1/7  -1/35
F   3000/7    2/7   15/7   10/7
iteration: 2
leading col, row: 2 3
   free    X1   A1     D
X2    4   3/5   -7   1/5
X3    8     0    1     0
Z    34  13/5  -22  -4/5
K   112  -1/5    4  -7/5
F   420    -1   15     1
[Fraction(-1, 7) Fraction(22, 7) Fraction(-4, 7) Fraction(-15, 7)]
iterations: 2


{'Z': 34, 'D': 0, 'K': 112, 'F': 420, 'X1': 0, 'X2': 4, 'X3': 8}

In [104]:
table = Xs
rows=['Z', 'D', 'K', 'F']
columns=['free', 'X1', 'X2', 'X3']

iteration = 0
columns = ['free'] + [f'X{i}' for i in range(1, table.shape[1])] if columns is None else columns
rows = [f'X{i + table.shape[1]}' for i in range(table.shape[0] - 1)] + ['F'] if rows is None else rows
table = np.array(table).copy()
table = table + Fraction()
prev_leading_col = None

start_table = pd.DataFrame(table, columns=columns, index=rows)
# max_iter = math.e ** np.sum(start_table.shape)

print(iteration)
print(start_table)

def step(table: np.ndarray, columns: List[str], rows: List[str], leading_row=None, leading_col=None, optimization=True) -> dict:
    global iteration, prev_leading_col

    iteration += 1
    print("iteration:", iteration)
    # print(pd.DataFrame(table, columns=columns, index=rows))

    table2 = table.copy()
    if prev_leading_col is not None:
        table2[0, prev_leading_col] = 0

    if leading_col is None:
        leading_col = np.where(table2[-1, 1:] <= 0)[0][0] + 1 if optimization and sum(table2[-1, 1:] <= 0) == 1 \
            else np.abs(table2[:-1, 1:]).argmax() % (table.shape[1] - 1) + 1

    prev_leading_col = leading_col

    if leading_row is None:
        col = np.divide(table2.T[0, :-1], table2.T[leading_col, :-1])
        col[np.logical_and(table2.T[0, :-1] >= 0, table2.T[leading_col, :-1] <= 0)] = np.inf
        leading_row = np.abs(col).argmin()

        # print("col:", col)

    table2[0] = table[leading_row] / table[leading_row, leading_col]
    table2[0, leading_col] = 1 / table[leading_row, leading_col]
    auxiliary = -table.T[leading_col]
    auxiliary = np.delete(auxiliary, leading_row)
    table = np.delete(table, leading_row, axis=0)
    table.T[leading_col] = 0

    for i in range(auxiliary.shape[0]):
        table2[i + 1] = table2[0] * auxiliary[i] + table[i]

    table = table2

    changed_col = columns[leading_col]
    columns[leading_col] = rows.pop(leading_row)
    rows.insert(0, changed_col)

    print("leading col, row:", leading_col, leading_row)
    print(pd.DataFrame(table, columns=columns, index=rows))
    print(auxiliary)

    return table

while np.sum(table[-1] >= 0) < table.shape[1]:
    table = step(table, columns, rows)

target_row: np.ndarray = table[np.isin(rows, start_table.columns)]

r(target_row)

0
  free  X1  X2   X3
Z  450   5   4   50
D  300   3   5   35
K  500   4   7   45
F    0  -4  -5  -50
iteration: 1
leading col, row: 3 1
      free    X1     X2      D
X3    60/7  3/35    1/7   1/35
Z    150/7   5/7  -22/7  -10/7
K    800/7   1/7    4/7   -9/7
F   3000/7   2/7   15/7   10/7
[Fraction(-50, 1) Fraction(-45, 1) Fraction(50, 1)]


array([[9, 0, 0, 0]])

In [50]:
while np.sum(table[-1] >= 0) < table.shape[1]:
    table = step(table, columns, rows)

iteration: 3
leading col, row: 1 0
          free            X2         A1             D
X1    6.666667  1.666667e+00 -11.666667  3.333333e-01
X3    8.000000  2.312965e-17   1.000000  4.625929e-18
Z    16.666667 -4.333333e+00   8.333333 -1.666667e+00
K   113.333333  3.333333e-01   1.666667 -1.333333e+00
F   426.666667  1.666667e+00   3.333333  1.333333e+00
[ 1.38777878e-17 -2.60000000e+00  2.00000000e-01  1.00000000e+00]


In [51]:
target_row = table[np.isin(rows, start_table.columns)]
target_row = target_row[np.abs(target_row[:, 0] - np.round(target_row[:, 0])) >= 1.e-14]
target_row = target_row - np.floor(target_row)
target_row = target_row[target_row[:, 0].argmax()]
target_row

array([0.66666667, 0.66666667, 0.33333333, 0.33333333])

In [52]:
rows.insert(len(rows) - 1, f'A{np.sum(table.shape) - np.sum(start_table.shape) + 1}')
table = np.insert(table, [len(rows) - 2], -target_row, axis=0)
print("iteration:", iteration + 0.5)
print(pd.DataFrame(table, columns=columns, index=rows))
table = step(table, columns, rows, optimization=False)

iteration: 3.5
          free            X2         A1             D
X1    6.666667  1.666667e+00 -11.666667  3.333333e-01
X3    8.000000  2.312965e-17   1.000000  4.625929e-18
Z    16.666667 -4.333333e+00   8.333333 -1.666667e+00
K   113.333333  3.333333e-01   1.666667 -1.333333e+00
A2   -0.666667 -6.666667e-01  -0.333333 -3.333333e-01
F   426.666667  1.666667e+00   3.333333  1.333333e+00
iteration: 4
leading col, row: 2 4
            free    X2    A2     D
A1  2.000000e+00   2.0  -3.0   1.0
X1  3.000000e+01  25.0 -35.0  12.0
X3  6.000000e+00  -2.0   3.0  -1.0
Z   2.415845e-13 -21.0  25.0 -10.0
K   1.100000e+02  -3.0   5.0  -3.0
F   4.200000e+02  -5.0  10.0  -2.0
[11.66666667 -1.         -8.33333333 -1.66666667 -3.33333333]


In [53]:
while np.sum(table[-1] >= 0) < table.shape[1]:
    table = step(table, columns, rows)

iteration: 5
leading col, row: 2 3
            free    X2     Z    D
A2  9.663381e-15 -0.84  0.04 -0.4
A1  2.000000e+00 -0.52  0.12 -0.2
X1  3.000000e+01 -4.40  1.40 -2.0
X3  6.000000e+00  0.52 -0.12  0.2
K   1.100000e+02  1.20 -0.20 -1.0
F   4.200000e+02  3.40 -0.40  2.0
[  3.  35.  -3.  -5. -10.]
iteration: 6
leading col, row: 2 1
          free            X2         A1             D
Z    16.666667 -4.333333e+00   8.333333 -1.666667e+00
A2   -0.666667 -6.666667e-01  -0.333333 -3.333333e-01
X1    6.666667  1.666667e+00 -11.666667  3.333333e-01
X3    8.000000 -2.220446e-16   1.000000 -1.110223e-16
K   113.333333  3.333333e-01   1.666667 -1.333333e+00
F   426.666667  1.666667e+00   3.333333  1.333333e+00
[-0.04 -1.4   0.12  0.2   0.4 ]


In [54]:
target_row = table[np.isin(rows, start_table.columns)]
target_row = target_row[np.abs(target_row[:, 0] - np.round(target_row[:, 0])) >= 1.e-14]
target_row = target_row - np.floor(target_row)
target_row = target_row[target_row[:, 0].argmax()]
target_row

array([0.66666667, 0.66666667, 0.33333333, 0.33333333])

In [55]:
rows.insert(len(rows) - 1, f'A{np.sum(table.shape) - np.sum(start_table.shape) + 1}')
table = np.insert(table, [len(rows) - 2], -target_row, axis=0)
print("iteration:", iteration + 0.5)
print(pd.DataFrame(table, columns=columns, index=rows))
table = step(table, columns, rows, optimization=False)

iteration: 6.5
          free            X2         A1             D
Z    16.666667 -4.333333e+00   8.333333 -1.666667e+00
A2   -0.666667 -6.666667e-01  -0.333333 -3.333333e-01
X1    6.666667  1.666667e+00 -11.666667  3.333333e-01
X3    8.000000 -2.220446e-16   1.000000 -1.110223e-16
K   113.333333  3.333333e-01   1.666667 -1.333333e+00
A3   -0.666667 -6.666667e-01  -0.333333 -3.333333e-01
F   426.666667  1.666667e+00   3.333333  1.333333e+00
iteration: 7
leading col, row: 2 1
            free            X2    A2             D
A1  2.000000e+00  2.000000e+00  -3.0  1.000000e+00
Z   2.380318e-13 -2.100000e+01  25.0 -1.000000e+01
X1  3.000000e+01  2.500000e+01 -35.0  1.200000e+01
X3  6.000000e+00 -2.000000e+00   3.0 -1.000000e+00
K   1.100000e+02 -3.000000e+00   5.0 -3.000000e+00
A3 -1.776357e-15 -4.440892e-16  -1.0  1.221245e-15
F   4.200000e+02 -5.000000e+00  10.0 -2.000000e+00
[-8.33333333 11.66666667 -1.         -1.66666667  0.33333333 -3.33333333]
